In [5]:
categories = ['alt.atheism','soc.religion.christian','comp.graphics', 'sci.med']

In [7]:
from sklearn.datasets import fetch_20newsgroups
twenty_train =fetch_20newsgroups(subset='train',categories=categories, shuffle= True, random_state=25)

In [9]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [11]:
len(twenty_train.data)

2257

In [12]:
len(twenty_train.filenames)

2257

In [13]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: Desiree_Bradley@mindlink.bc.ca (Desiree Bradley)
Subject: Being right about messiahs
Organization: MIND LINK! - British Columbia, Canada


In [15]:
print(twenty_train.target_names[twenty_train.target[0]])

soc.religion.christian


In [17]:
twenty_train.target[:10]

array([3, 2, 3, 0, 1, 1, 1, 1, 1, 0])

In [18]:
for t in twenty_train.target[:10]:
  print(twenty_train.target_names[t])

soc.religion.christian
sci.med
soc.religion.christian
alt.atheism
comp.graphics
comp.graphics
comp.graphics
comp.graphics
comp.graphics
alt.atheism


Extracting feature from text files

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x_train_count = count_vect.fit_transform(twenty_train.data)
x_train_count.shape

(2257, 35788)

In [21]:
count_vect.vocabulary_.get(u'algorithm')

4690

**From occurences to frequencies**
tf and tf-idf calculation

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer= TfidfTransformer(use_idf=False).fit(x_train_count)
x_train_tf = tf_transformer.transform(x_train_count)
x_train_tf.shape

(2257, 35788)

In [25]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_count)
x_train_tfidf.shape

(2257, 35788)

Training Classifier

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf, twenty_train.target)

In [27]:
from sre_constants import CATEGORY
docs_new= ['God is love', 'OpenGL on the GPU is fast']
x_new_counts = count_vect.transform(docs_new)
x_new_tfidf = tfidf_transformer.transform(x_new_counts)


predicted = clf.predict(x_new_tfidf)


for doc, category in zip(docs_new, predicted):
  print('%r => %s'% (doc, twenty_train.target_names[category]))
   

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


Building Pipeline

In [28]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf',MultinomialNB())
])


In [30]:
text_clf.fit(twenty_train.data, twenty_train.target)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

Evaluvation of performance on test set

In [32]:
import numpy as np
twenty_test = fetch_20newsgroups(subset = 'test',
                                 categories=categories, shuffle= True, random_state=25)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean( predicted == twenty_test.target)

0.8348868175765646

using SVM


In [35]:
from sklearn import pipeline
from sklearn.linear_model import SGDClassifier
test_clf = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'hinge', penalty= '12',
                          alpha =1e-3, random_state= 25,
                          max_iter = 5, tol = None)),
])

text_clf.fit(twenty_train.data,twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

In [38]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
                                    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



In [39]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]])